<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/333_SalesEnab_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# ⏰ Follow-Up Tracking Utilities

**Preventing silent revenue loss through rules-based accountability**

This module solves one of the most expensive and least visible problems in sales:

> Deals don’t usually fail dramatically — they fail quietly when follow-ups slip.

Instead of hoping reps remember every commitment, this utility **systematically tracks promises, deadlines, and urgency**, using simple, transparent rules.

No AI guessing.
No subjective reminders.
Just operational discipline.

---

## Why This Matters

Sales leaders consistently underestimate how much revenue is lost to:

* missed follow-ups
* delayed responses
* “I thought I already sent that”

This utility turns follow-ups into:

* explicit objects
* measurable timelines
* enforceable priorities

That’s how you move from *best effort* to **process reliability**.

---

## 🧱 Date & Time Handling (Foundation First)

### `parse_datetime`

This function does one thing well:

* safely converts real-world timestamps into usable time objects

Why this matters:

* CRM data is messy
* timezones are inconsistent
* reliability matters more than elegance

The utility gracefully handles:

* UTC timestamps (`Z`)
* timezone-aware values
* fallback parsing

This ensures the agent **never crashes because of time formatting**, which is critical in production systems.

---

### `days_since`

This function translates timestamps into **business-relevant time**:

> “How long has it been since we last interacted?”

Why this matters:

* Time drives urgency
* Urgency drives action
* Action drives revenue

This abstraction lets the rest of the system reason in **days**, not timestamps — exactly how humans think.

---

## 🔍 `extract_follow_up_actions`: The Heart of the System

This function scans interaction history and answers a simple but powerful question:

> *“What did we promise — and did we do it?”*

### What it looks for

* An interaction included a `next_step_promised`
* That step has **not** been completed
* Enough time has passed to matter

Only those interactions become follow-up actions.

This prevents noise and focuses attention on **real commitments**.

---

## 🟡 Status Classification (Clear, Explainable Rules)

Each follow-up is classified into one of three states:

* **pending** → still within acceptable time
* **due_soon** → approaching the deadline
* **overdue** → commitment missed

These statuses are based on **explicit thresholds**, not intuition.

Why this matters:

* Everyone agrees on what “overdue” means
* Escalation is consistent
* No one feels unfairly singled out

---

## 🚦 Priority Assignment (Actionable, Not Passive)

Status drives priority automatically:

* overdue → **high**
* due soon → **medium**
* pending → **low**

This makes follow-ups **operational**, not informational.

The agent isn’t just saying “this exists” — it’s saying:

> “This needs attention *now*.”

---

## 📅 Due Date Estimation (Lightweight, Practical)

The system assumes:

* follow-ups should generally happen within 2 days

This is intentionally simple.

Why?

* It reflects common sales expectations
* It avoids over-engineering
* It keeps urgency aligned with reality

Executives care far more about **consistency** than perfect precision.

---

## 🧠 Action Type Inference (`_infer_action_type`)

This helper function translates vague promises into **structured action types**:

* send pricing
* schedule a call
* send a proposal
* schedule a demo

Why this matters:

* Follow-ups become automatable
* Nudges become specific
* Reporting becomes structured

This is how you move from:

> “You should follow up”

to:

> “Send pricing to Lead X today.”

---

## 🔴 Targeted Views: Overdue & Due-Soon Filters

The final helpers:

```python
get_overdue_follow_ups
get_due_soon_follow_ups
```

These are deceptively powerful.

They enable:

* rep-specific nudges
* manager dashboards
* SLA-style reporting
* executive summaries

Instead of dumping data, the agent delivers **focused attention lists**.

---

## Why This Utility Is Architecturally Strong

This module demonstrates:

* 🧠 **Rules-based accountability**
* 🔍 **Observable commitments**
* ⏱ **Time-aware urgency**
* 📊 **Measurable process hygiene**
* 💰 **Direct revenue protection**

Most agents help you *write messages*.
This one helps you **keep promises**.

That’s far more valuable.

---

## How This Fits the Orchestrator

This utility feeds directly into:

* rep nudging
* deal risk detection
* executive reporting
* performance metrics

It also reinforces your core philosophy:

> *Trust comes from consistency, not cleverness.*



# Follow-up Tracking Utilities for Sales Enablement Orchestrator

In [ ]:
"""Follow-up Tracking Utilities for Sales Enablement Orchestrator

This module contains utilities for tracking follow-up actions and detecting overdue items.
"""

from typing import Dict, List, Any, Optional
from datetime import datetime, timedelta


def parse_datetime(dt_string: str) -> datetime:
    """
    Parse ISO 8601 datetime string.

    Args:
        dt_string: ISO 8601 datetime string (e.g., "2025-12-10T14:30:00Z")

    Returns:
        datetime object
    """
    # Handle Z suffix (UTC)
    if dt_string.endswith('Z'):
        dt_string = dt_string[:-1] + '+00:00'

    # Try parsing with timezone
    try:
        return datetime.fromisoformat(dt_string)
    except ValueError:
        # Fallback: try without timezone
        return datetime.fromisoformat(dt_string.replace('Z', ''))


def days_since(dt_string: str) -> float:
    """
    Calculate days since a datetime string.

    Args:
        dt_string: ISO 8601 datetime string

    Returns:
        Number of days (can be fractional)
    """
    try:
        dt = parse_datetime(dt_string)
        now = datetime.now(dt.tzinfo) if dt.tzinfo else datetime.now()
        delta = now - dt
        return delta.total_seconds() / (24 * 3600)
    except (ValueError, AttributeError):
        return 0.0


def extract_follow_up_actions(
    interactions: List[Dict[str, Any]],
    follow_up_overdue_days: int = 3
) -> List[Dict[str, Any]]:
    """
    Extract follow-up actions from interactions.

    Identifies interactions where next_step_promised exists but next_step_completed is False.

    Args:
        interactions: List of interaction dictionaries
        follow_up_overdue_days: Days past interaction to flag as overdue

    Returns:
        List of follow-up action dictionaries
    """
    follow_ups = []

    for interaction in interactions:
        next_step_promised = interaction.get("next_step_promised")
        next_step_completed = interaction.get("next_step_completed", False)

        # Only track incomplete follow-ups
        if next_step_promised and not next_step_completed:
            interaction_datetime = interaction.get("datetime")
            lead_id = interaction.get("lead_id")
            rep_id = interaction.get("rep_id")
            interaction_id = interaction.get("interaction_id")

            if not interaction_datetime or not lead_id:
                continue

            days_since_interaction = days_since(interaction_datetime)

            # Determine status
            if days_since_interaction > follow_up_overdue_days:
                status = "overdue"
            elif days_since_interaction > (follow_up_overdue_days - 1):
                status = "due_soon"
            else:
                status = "pending"

            # Determine priority
            if status == "overdue":
                priority = "high"
            elif status == "due_soon":
                priority = "medium"
            else:
                priority = "low"

            # Estimate due date (assume follow-up should happen within 2 days of interaction)
            try:
                interaction_dt = parse_datetime(interaction_datetime)
                due_date = (interaction_dt + timedelta(days=2)).strftime("%Y-%m-%d")
            except (ValueError, AttributeError):
                due_date = None

            follow_up = {
                "lead_id": lead_id,
                "rep_id": rep_id,
                "interaction_id": interaction_id,
                "action_type": _infer_action_type(next_step_promised),
                "next_step_promised": next_step_promised,
                "promised_in_interaction": interaction_id,
                "interaction_datetime": interaction_datetime,
                "days_since_interaction": round(days_since_interaction, 1),
                "due_date": due_date,
                "status": status,
                "priority": priority
            }

            follow_ups.append(follow_up)

    return follow_ups


def _infer_action_type(next_step_promised: str) -> str:
    """
    Infer action type from next_step_promised string.

    Args:
        next_step_promised: String describing the next step

    Returns:
        Action type: "send_case_study" | "schedule_call" | "send_pricing" | "other"
    """
    next_step_lower = next_step_promised.lower()

    if "case study" in next_step_lower or "case study" in next_step_lower:
        return "send_case_study"
    elif "schedule" in next_step_lower or "call" in next_step_lower or "meeting" in next_step_lower:
        return "schedule_call"
    elif "pricing" in next_step_lower:
        return "send_pricing"
    elif "proposal" in next_step_lower:
        return "send_proposal"
    elif "demo" in next_step_lower:
        return "schedule_demo"
    else:
        return "other"


def get_overdue_follow_ups(
    follow_up_actions: List[Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Get follow-up actions that are overdue.

    Args:
        follow_up_actions: List of follow-up action dictionaries

    Returns:
        List of overdue follow-up actions
    """
    return [f for f in follow_up_actions if f.get("status") == "overdue"]


def get_due_soon_follow_ups(
    follow_up_actions: List[Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Get follow-up actions that are due soon.

    Args:
        follow_up_actions: List of follow-up action dictionaries

    Returns:
        List of due soon follow-up actions
    """
    return [f for f in follow_up_actions if f.get("status") == "due_soon"]





# 🧭 Rep Nudging Utilities

**Turning signals into timely, actionable intervention**

This module defines **when and how the agent is allowed to interrupt sales reps**.

That’s an important distinction.

Most systems either:

* spam reminders, or
* stay silent until it’s too late

This nudging framework sits in the middle: **intervening only when there’s a clear, explainable reason**.

---

## Why This Matters

Sales productivity is not about more activity — it’s about **timely activity**.

This module ensures that:

* promises are kept
* deals don’t stall unnoticed
* high-value leads aren’t ignored
* risk is surfaced early

And it does so using **rules leadership can inspect and control**.

---

## 🔔 Nudge Types (Each One Has a Business Reason)

This module supports **four distinct nudge categories**, each tied to a specific operational failure mode.

---

### 1️⃣ Follow-Up Nudges

*“We promised something — and time has passed.”*

```python
generate_follow_up_nudge(...)
```

This nudge is triggered when:

* a next step was explicitly promised
* it hasn’t been completed
* enough time has elapsed to matter

The message includes:

* the customer’s company name
* what was promised
* how long ago the promise was made

Why this matters:

* Accountability is based on commitments, not assumptions
* The nudge is factual, not accusatory
* Reps immediately know what action to take

This is how you prevent **quiet deal decay**.

---

### 2️⃣ Stalled Deal Nudges

*“The deal stopped moving — why?”*

```python
generate_stalled_deal_nudge(...)
```

This nudge fires when a deal:

* sits in the same stage too long
* represents meaningful revenue
* lacks forward momentum

The message:

* names the deal
* shows deal value
* states how long it’s been stalled
* prompts review or advancement

Why this matters:

* Pipeline hygiene becomes visible
* Stagnation is addressed proactively
* Managers don’t have to hunt for problems

This is early intervention — not post-mortem analysis.

---

### 3️⃣ High-Priority Lead Nudges

*“This lead matters — and no one has acted yet.”*

```python
generate_high_priority_lead_nudge(...)
```

This nudge targets leads that:

* score highly on your prioritization model
* have no recent interactions
* represent unrealized opportunity

The message:

* highlights priority score
* reinforces recommended action
* applies urgency based on score strength

Why this matters:

* Attention is redirected to where ROI is highest
* High-value leads don’t fall through the cracks
* Reps are guided, not micromanaged

This is **attention allocation**, not pressure.

---

### 4️⃣ Deal-at-Risk Nudges

*“There are warning signs — act now.”*

```python
generate_deal_at_risk_nudge(...)
```

This is the strongest intervention.

It activates when:

* a deal has risk flags
* time-in-stage is excessive
* recent interactions show negative sentiment

The agent assembles a **risk summary** in plain language:

* stalled duration
* known risk flags
* sentiment issues

Why this matters:

* Risk is explained, not just flagged
* Reps understand *why* escalation happened
* Intervention is justified and credible

This is exactly how experienced managers operate — now systematized.

---

## 🧠 Central Orchestration: `generate_all_rep_nudges`

This function is the **policy enforcement layer**.

It decides:

* which nudges are allowed
* under what conditions
* based on configurable leadership preferences

Key design choice:

> Nudging is **opt-in by category**, not hardcoded.

```python
config.nudge_follow_up_overdue
config.nudge_stalled_deals
config.nudge_high_priority_leads
```

This gives leadership control over:

* interruption volume
* cultural fit
* rollout pacing

---

## 🛑 Noise Control: No Nudges Without Cause

Notice what this system *doesn’t* do:

* It doesn’t nudge if there was recent interaction
* It doesn’t nudge all high-priority leads blindly
* It doesn’t assume urgency without evidence

This avoids the #1 failure mode of agent systems:

> *Alert fatigue.*

Every nudge has:

* a trigger
* a reason
* a recommended action

---

## 🧩 Why This Design Is Enterprise-Grade

This nudging system demonstrates:

* 🧠 **Judgment, not automation**
* 🎯 **Intervention tied to business risk**
* 🎛️ **Executive control over behavior**
* 📊 **Measurable performance impact**
* 🔍 **Explainable decisions**

Most agent nudges feel like suggestions.
These feel like **management signals**.

---

## How This Supports ROI & Trust

This module directly supports:

* higher close rates
* shorter sales cycles
* fewer dropped deals
* better rep focus
* manager leverage at scale

And it does so **without replacing humans** — it supports them.

---

## How This Fits the Orchestrator Vision

This nudging layer is the connective tissue between:

* lead prioritization
* follow-up tracking
* deal risk detection
* executive reporting

It’s where **insight turns into action**.



# Rep Nudging Utilities for Sales Enablement Orchestrator

In [ ]:
"""Rep Nudging Utilities for Sales Enablement Orchestrator

This module contains utilities for generating nudges for sales reps.
"""

from typing import Dict, List, Any, Optional


def generate_follow_up_nudge(
    follow_up: Dict[str, Any],
    lead: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """
    Generate a nudge for an overdue or due soon follow-up.

    Args:
        follow_up: Follow-up action dictionary
        lead: Optional lead dictionary for context

    Returns:
        Nudge dictionary
    """
    lead_id = follow_up.get("lead_id")
    rep_id = follow_up.get("rep_id")
    action_type = follow_up.get("action_type", "other")
    next_step_promised = follow_up.get("next_step_promised", "")
    days_since = follow_up.get("days_since_interaction", 0)
    status = follow_up.get("status", "pending")

    company_name = lead.get("company_name", "lead") if lead else "lead"

    # Generate message based on status
    if status == "overdue":
        message = f"⚠️ Follow-up with {company_name} ({lead_id}) is overdue. You promised to {next_step_promised} {int(days_since)} days ago."
    elif status == "due_soon":
        message = f"📅 Follow-up with {company_name} ({lead_id}) is due soon. You promised to {next_step_promised} {int(days_since)} days ago."
    else:
        message = f"📋 Follow-up reminder: {company_name} ({lead_id}) - {next_step_promised}"

    return {
        "rep_id": rep_id,
        "nudge_type": "follow_up_due",
        "lead_id": lead_id,
        "deal_id": None,
        "message": message,
        "urgency": follow_up.get("priority", "medium"),
        "recommended_action": next_step_promised
    }


def generate_stalled_deal_nudge(
    deal: Dict[str, Any],
    lead: Optional[Dict[str, Any]] = None
) -> Dict[str, Any]:
    """
    Generate a nudge for a stalled deal.

    Args:
        deal: Deal dictionary
        lead: Optional lead dictionary for context

    Returns:
        Nudge dictionary
    """
    lead_id = deal.get("lead_id")
    rep_id = deal.get("rep_id")
    deal_id = deal.get("deal_id")
    stage = deal.get("stage", "")
    days_in_stage = deal.get("days_in_stage", 0)
    deal_value = deal.get("deal_value_usd", 0)

    company_name = lead.get("company_name", "lead") if lead else "lead"

    message = f"⏸️ Deal {deal_id} ({company_name}, ${deal_value:,.0f}) has been in {stage} stage for {days_in_stage} days. Consider advancing or addressing blockers."

    return {
        "rep_id": rep_id,
        "nudge_type": "stalled_deal",
        "lead_id": lead_id,
        "deal_id": deal_id,
        "message": message,
        "urgency": "high" if days_in_stage > 25 else "medium",
        "recommended_action": "review deal status and advance to next stage"
    }


def generate_high_priority_lead_nudge(
    lead: Dict[str, Any],
    rep_id: Optional[str] = None
) -> Dict[str, Any]:
    """
    Generate a nudge for a high-priority lead with no recent interaction.

    Args:
        lead: Lead dictionary (should have priority_score)
        rep_id: Optional rep_id to assign

    Returns:
        Nudge dictionary
    """
    lead_id = lead.get("lead_id")
    company_name = lead.get("company_name", "lead")
    priority_score = lead.get("priority_score", 0.0)
    recommended_action = lead.get("recommended_action", "initiate outreach")

    message = f"⭐ High-priority lead: {company_name} ({lead_id}) has priority score {priority_score:.1f}. {recommended_action}."

    return {
        "rep_id": rep_id,
        "nudge_type": "high_priority_lead",
        "lead_id": lead_id,
        "deal_id": None,
        "message": message,
        "urgency": "high" if priority_score > 80 else "medium",
        "recommended_action": recommended_action
    }


def generate_deal_at_risk_nudge(
    deal: Dict[str, Any],
    lead: Optional[Dict[str, Any]] = None,
    interactions: Optional[List[Dict[str, Any]]] = None
) -> Dict[str, Any]:
    """
    Generate a nudge for a deal at risk.

    Args:
        deal: Deal dictionary
        lead: Optional lead dictionary for context
        interactions: Optional list of interactions for this lead

    Returns:
        Nudge dictionary
    """
    lead_id = deal.get("lead_id")
    rep_id = deal.get("rep_id")
    deal_id = deal.get("deal_id")
    deal_value = deal.get("deal_value_usd", 0)
    risk_flags = deal.get("risk_flags", [])
    days_in_stage = deal.get("days_in_stage", 0)

    company_name = lead.get("company_name", "lead") if lead else "lead"

    # Check for negative sentiment in recent interactions
    has_negative_sentiment = False
    if interactions:
        recent_interactions = [i for i in interactions if i.get("sentiment") == "negative"]
        if recent_interactions:
            has_negative_sentiment = True

    risk_factors = []
    if days_in_stage > 25:
        risk_factors.append(f"stalled for {days_in_stage} days")
    if risk_flags:
        risk_factors.append(", ".join(risk_flags))
    if has_negative_sentiment:
        risk_factors.append("negative sentiment in recent interactions")

    risk_summary = " | ".join(risk_factors) if risk_factors else "multiple risk factors"

    message = f"🚨 Deal at risk: {deal_id} ({company_name}, ${deal_value:,.0f}) - {risk_summary}. Immediate attention needed."

    return {
        "rep_id": rep_id,
        "nudge_type": "deal_at_risk",
        "lead_id": lead_id,
        "deal_id": deal_id,
        "message": message,
        "urgency": "high",
        "recommended_action": "review deal and address risk factors"
    }


def generate_all_rep_nudges(
    follow_up_actions: List[Dict[str, Any]],
    stalled_deals: List[Dict[str, Any]],
    at_risk_deals: List[Dict[str, Any]],
    top_priority_leads: List[Dict[str, Any]],
    leads_lookup: Dict[str, Dict[str, Any]],
    interactions_lookup: Dict[str, List[Dict[str, Any]]],
    config: Any
) -> List[Dict[str, Any]]:
    """
    Generate all rep nudges based on various triggers.

    Args:
        follow_up_actions: List of follow-up actions
        stalled_deals: List of stalled deals
        at_risk_deals: List of at-risk deals
        top_priority_leads: List of top priority leads
        leads_lookup: Dictionary mapping lead_id to lead dict
        interactions_lookup: Dictionary mapping lead_id to list of interactions
        config: Configuration object with nudge settings

    Returns:
        List of nudge dictionaries
    """
    nudges = []

    # Follow-up nudges
    if config.nudge_follow_up_overdue:
        overdue_follow_ups = [f for f in follow_up_actions if f.get("status") == "overdue"]
        for follow_up in overdue_follow_ups:
            lead_id = follow_up.get("lead_id")
            lead = leads_lookup.get(lead_id)
            nudge = generate_follow_up_nudge(follow_up, lead)
            nudges.append(nudge)

    # Stalled deal nudges
    if config.nudge_stalled_deals:
        for deal in stalled_deals:
            lead_id = deal.get("lead_id")
            lead = leads_lookup.get(lead_id)
            nudge = generate_stalled_deal_nudge(deal, lead)
            nudges.append(nudge)

    # High-priority lead nudges (for leads with no recent interactions)
    if config.nudge_high_priority_leads:
        for lead in top_priority_leads:
            lead_id = lead.get("lead_id")
            interactions = interactions_lookup.get(lead_id, [])

            # Only nudge if no recent interactions (last 7 days)
            has_recent_interaction = False
            if interactions:
                from agents.sales_enablement.utilities.follow_up_tracking import days_since
                for interaction in interactions:
                    interaction_dt = interaction.get("datetime")
                    if interaction_dt and days_since(interaction_dt) < 7:
                        has_recent_interaction = True
                        break

            if not has_recent_interaction:
                # Assign to rep if deal exists, otherwise None
                deal = None  # Will be looked up if needed
                rep_id = lead.get("rep_id")  # Could be assigned based on region, etc.
                nudge = generate_high_priority_lead_nudge(lead, rep_id)
                nudges.append(nudge)

    # At-risk deal nudges
    for deal in at_risk_deals:
        lead_id = deal.get("lead_id")
        lead = leads_lookup.get(lead_id)
        interactions = interactions_lookup.get(lead_id, [])
        nudge = generate_deal_at_risk_nudge(deal, lead, interactions)
        nudges.append(nudge)

    return nudges



# Test Phase 4: Follow-up Coordination & Rep Nudging Utilities

In [ ]:
"""Test Phase 4: Follow-up Coordination & Rep Nudging Utilities

Test utilities independently before building nodes.
"""

import sys
from pathlib import Path

# Add project root to path
project_root = Path(__file__).parent
sys.path.insert(0, str(project_root))

from agents.sales_enablement.utilities.follow_up_tracking import (
    extract_follow_up_actions,
    get_overdue_follow_ups,
    get_due_soon_follow_ups,
    days_since
)
from agents.sales_enablement.utilities.rep_nudging import (
    generate_follow_up_nudge,
    generate_stalled_deal_nudge,
    generate_high_priority_lead_nudge,
    generate_deal_at_risk_nudge
)
from agents.sales_enablement.utilities.data_loading import (
    load_interactions,
    load_deals,
    load_leads
)
from config import SalesEnablementOrchestratorConfig


def test_extract_follow_up_actions():
    """Test extracting follow-up actions from interactions"""
    print("Testing extract_follow_up_actions...")

    config = SalesEnablementOrchestratorConfig()
    interactions = load_interactions(config.data_dir, config.interactions_file)

    follow_ups = extract_follow_up_actions(interactions, follow_up_overdue_days=3)

    assert isinstance(follow_ups, list), "should return list"
    assert len(follow_ups) > 0, "should find some follow-ups"
    assert "lead_id" in follow_ups[0], "should have lead_id"
    assert "status" in follow_ups[0], "should have status"
    assert "priority" in follow_ups[0], "should have priority"

    # Check status distribution
    overdue = [f for f in follow_ups if f["status"] == "overdue"]
    due_soon = [f for f in follow_ups if f["status"] == "due_soon"]
    pending = [f for f in follow_ups if f["status"] == "pending"]

    print(f"✅ extract_follow_up_actions test passed!")
    print(f"   Total follow-ups: {len(follow_ups)}")
    print(f"   Overdue: {len(overdue)}, Due soon: {len(due_soon)}, Pending: {len(pending)}")
    if follow_ups:
        print(f"   Sample: {follow_ups[0]['lead_id']} - {follow_ups[0]['action_type']} ({follow_ups[0]['status']})")
    print()

    return follow_ups


def test_get_overdue_follow_ups(follow_ups):
    """Test getting overdue follow-ups"""
    print("Testing get_overdue_follow_ups...")

    overdue = get_overdue_follow_ups(follow_ups)

    assert isinstance(overdue, list), "should return list"
    assert all(f.get("status") == "overdue" for f in overdue), "all should be overdue"

    print(f"✅ get_overdue_follow_ups test passed!")
    print(f"   Overdue follow-ups: {len(overdue)}")
    print()


def test_generate_follow_up_nudge():
    """Test generating follow-up nudge"""
    print("Testing generate_follow_up_nudge...")

    follow_up = {
        "lead_id": "L-001",
        "rep_id": "SR-01",
        "action_type": "send_pricing",
        "next_step_promised": "send pricing comparison",
        "days_since_interaction": 5.0,
        "status": "overdue",
        "priority": "high"
    }

    lead = {
        "lead_id": "L-001",
        "company_name": "Northstar Logistics"
    }

    nudge = generate_follow_up_nudge(follow_up, lead)

    assert "rep_id" in nudge, "should have rep_id"
    assert "nudge_type" in nudge, "should have nudge_type"
    assert nudge["nudge_type"] == "follow_up_due", "should be follow_up_due type"
    assert "message" in nudge, "should have message"
    assert "urgency" in nudge, "should have urgency"

    print(f"✅ generate_follow_up_nudge test passed!")
    print(f"   Nudge: {nudge['message'][:60]}...")
    print()


def test_generate_stalled_deal_nudge():
    """Test generating stalled deal nudge"""
    print("Testing generate_stalled_deal_nudge...")

    deal = {
        "deal_id": "D-001",
        "lead_id": "L-001",
        "rep_id": "SR-01",
        "stage": "Proposal",
        "days_in_stage": 22,
        "deal_value_usd": 75000
    }

    lead = {
        "lead_id": "L-001",
        "company_name": "Northstar Logistics"
    }

    nudge = generate_stalled_deal_nudge(deal, lead)

    assert nudge["nudge_type"] == "stalled_deal", "should be stalled_deal type"
    assert "deal_id" in nudge, "should have deal_id"
    assert "message" in nudge, "should have message"

    print(f"✅ generate_stalled_deal_nudge test passed!")
    print(f"   Nudge: {nudge['message'][:60]}...")
    print()


def test_generate_high_priority_lead_nudge():
    """Test generating high-priority lead nudge"""
    print("Testing generate_high_priority_lead_nudge...")

    lead = {
        "lead_id": "L-020",
        "company_name": "Orion Aerospace",
        "priority_score": 86.6,
        "recommended_action": "assign senior rep and initiate outreach"
    }

    nudge = generate_high_priority_lead_nudge(lead, rep_id="SR-01")

    assert nudge["nudge_type"] == "high_priority_lead", "should be high_priority_lead type"
    assert "lead_id" in nudge, "should have lead_id"
    assert "message" in nudge, "should have message"

    print(f"✅ generate_high_priority_lead_nudge test passed!")
    print(f"   Nudge: {nudge['message'][:60]}...")
    print()


def test_generate_deal_at_risk_nudge():
    """Test generating deal at risk nudge"""
    print("Testing generate_deal_at_risk_nudge...")

    deal = {
        "deal_id": "D-004",
        "lead_id": "L-005",
        "rep_id": "SR-03",
        "deal_value_usd": 92000,
        "days_in_stage": 21,
        "risk_flags": ["discount pressure"]
    }

    lead = {
        "lead_id": "L-005",
        "company_name": "Skyline Retail Group"
    }

    interactions = [
        {
            "interaction_id": "INT-006",
            "sentiment": "negative",
            "key_topics": ["pricing concerns"]
        }
    ]

    nudge = generate_deal_at_risk_nudge(deal, lead, interactions)

    assert nudge["nudge_type"] == "deal_at_risk", "should be deal_at_risk type"
    assert "deal_id" in nudge, "should have deal_id"
    assert nudge["urgency"] == "high", "should be high urgency"

    print(f"✅ generate_deal_at_risk_nudge test passed!")
    print(f"   Nudge: {nudge['message'][:60]}...")
    print()


if __name__ == "__main__":
    print("=" * 60)
    print("Phase 4: Follow-up Coordination & Rep Nudging - Test Suite")
    print("=" * 60)
    print()

    follow_ups = test_extract_follow_up_actions()
    test_get_overdue_follow_ups(follow_ups)
    test_generate_follow_up_nudge()
    test_generate_stalled_deal_nudge()
    test_generate_high_priority_lead_nudge()
    test_generate_deal_at_risk_nudge()

    print("=" * 60)
    print("✅ All Phase 4 utility tests passed!")
    print("=" * 60)



# Deal analysis Utilities for Sales Enablement Orchestrator

In [ ]:
"""Deal Analysis Utilities for Sales Enablement Orchestrator

This module contains utilities for analyzing deals, detecting risks, and identifying opportunities.
"""

from typing import Dict, List, Any, Optional


def detect_stalled_deals(
    deals: List[Dict[str, Any]],
    stalled_deal_days: int = 20
) -> List[Dict[str, Any]]:
    """
    Detect deals that are stalled (in same stage for too long).

    Args:
        deals: List of deal dictionaries
        stalled_deal_days: Days in stage threshold for stalled

    Returns:
        List of stalled deal dictionaries with insight metadata
    """
    stalled = []

    for deal in deals:
        status = deal.get("status", "active")
        stage = deal.get("stage", "")
        days_in_stage = deal.get("days_in_stage", 0)

        # Only check active deals (not closed won/lost)
        if status != "active":
            continue

        # Check if stalled
        if days_in_stage > stalled_deal_days:
            insight = {
                "deal_id": deal.get("deal_id"),
                "lead_id": deal.get("lead_id"),
                "insight_type": "stalled",
                "risk_factors": [f"days_in_stage > {stalled_deal_days} ({days_in_stage} days)"],
                "opportunity_factors": [],
                "recommended_action": "review deal status and advance to next stage",
                "urgency": "high" if days_in_stage > 25 else "medium",
                "stage": stage,
                "days_in_stage": days_in_stage
            }
            stalled.append(insight)

    return stalled


def detect_at_risk_deals(
    deals: List[Dict[str, Any]],
    interactions_lookup: Dict[str, List[Dict[str, Any]]],
    at_risk_days_threshold: int = 25,
    at_risk_sentiment_threshold: float = 0.3
) -> List[Dict[str, Any]]:
    """
    Detect deals at risk of loss.

    Args:
        deals: List of deal dictionaries
        interactions_lookup: Dictionary mapping lead_id to list of interactions
        at_risk_days_threshold: Days in stage threshold
        at_risk_sentiment_threshold: Sentiment score threshold (below this is risk)

    Returns:
        List of at-risk deal dictionaries with insight metadata
    """
    at_risk = []

    for deal in deals:
        status = deal.get("status", "active")
        lead_id = deal.get("lead_id")
        days_in_stage = deal.get("days_in_stage", 0)
        risk_flags = deal.get("risk_flags", [])

        # Only check active deals
        if status != "active":
            continue

        risk_factors = []

        # Check days in stage
        if days_in_stage > at_risk_days_threshold:
            risk_factors.append(f"days_in_stage > {at_risk_days_threshold} ({days_in_stage} days)")

        # Check risk flags
        if risk_flags:
            risk_factors.extend(risk_flags)

        # Check sentiment in recent interactions
        interactions = interactions_lookup.get(lead_id, [])
        if interactions:
            negative_interactions = [i for i in interactions if i.get("sentiment") == "negative"]
            if negative_interactions:
                risk_factors.append("negative sentiment in recent interactions")

        # If we have risk factors, mark as at risk
        if risk_factors:
            insight = {
                "deal_id": deal.get("deal_id"),
                "lead_id": lead_id,
                "insight_type": "at_risk",
                "risk_factors": risk_factors,
                "opportunity_factors": [],
                "recommended_action": "review deal and address risk factors immediately",
                "urgency": "high",
                "stage": deal.get("stage", ""),
                "days_in_stage": days_in_stage
            }
            at_risk.append(insight)

    return at_risk


def detect_deal_opportunities(
    deals: List[Dict[str, Any]],
    interactions_lookup: Dict[str, List[Dict[str, Any]]],
    signals_lookup: Dict[str, Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Detect deal opportunities (positive signals).

    Args:
        deals: List of deal dictionaries
        interactions_lookup: Dictionary mapping lead_id to list of interactions
        signals_lookup: Dictionary mapping lead_id to signal dict

    Returns:
        List of opportunity insight dictionaries
    """
    opportunities = []

    for deal in deals:
        status = deal.get("status", "active")
        lead_id = deal.get("lead_id")

        # Only check active deals
        if status != "active":
            continue

        opportunity_factors = []

        # Check interactions for positive signals
        interactions = interactions_lookup.get(lead_id, [])
        if interactions:
            recent_interactions = interactions[-3:]  # Last 3 interactions
            positive_count = sum(1 for i in recent_interactions if i.get("sentiment") == "positive")

            if positive_count >= 2:
                opportunity_factors.append("multiple positive interactions")

            # Check for pricing discussion (buying signal)
            for interaction in recent_interactions:
                key_topics = interaction.get("key_topics", [])
                if "pricing" in key_topics:
                    opportunity_factors.append("pricing discussed")
                    break

        # Check signals for engagement
        signal = signals_lookup.get(lead_id)
        if signal:
            engagement_score = signal.get("engagement_score", 0.0)
            if engagement_score > 0.7:
                opportunity_factors.append("high engagement score")

        # If we have opportunity factors, create insight
        if opportunity_factors:
            insight = {
                "deal_id": deal.get("deal_id"),
                "lead_id": lead_id,
                "insight_type": "opportunity",
                "risk_factors": [],
                "opportunity_factors": opportunity_factors,
                "recommended_action": "advance deal to next stage",
                "urgency": "medium",
                "stage": deal.get("stage", ""),
                "days_in_stage": deal.get("days_in_stage", 0)
            }
            opportunities.append(insight)

    return opportunities


def generate_all_deal_insights(
    deals: List[Dict[str, Any]],
    interactions_lookup: Dict[str, List[Dict[str, Any]]],
    signals_lookup: Dict[str, Dict[str, Any]],
    config: Any
) -> List[Dict[str, Any]]:
    """
    Generate all deal insights (stalled, at-risk, opportunities).

    Args:
        deals: List of deal dictionaries
        interactions_lookup: Dictionary mapping lead_id to list of interactions
        signals_lookup: Dictionary mapping lead_id to signal dict
        config: Configuration object with thresholds

    Returns:
        List of all deal insight dictionaries
    """
    all_insights = []

    # Detect stalled deals
    stalled = detect_stalled_deals(deals, config.stalled_deal_days)
    all_insights.extend(stalled)

    # Detect at-risk deals
    at_risk = detect_at_risk_deals(
        deals,
        interactions_lookup,
        config.at_risk_days_threshold,
        config.at_risk_sentiment_threshold
    )
    all_insights.extend(at_risk)

    # Detect opportunities
    opportunities = detect_deal_opportunities(deals, interactions_lookup, signals_lookup)
    all_insights.extend(opportunities)

    return all_insights



# Sales Enablement Orchestrator Nodes

In [ ]:
"""Sales Enablement Orchestrator Nodes

This module contains all nodes for the Sales Enablement Orchestrator workflow.
Following the MVP-first approach: all nodes are rule-based (no LLM) for Phase 1-7.
"""

import json
from typing import Dict, Any
from config import SalesEnablementOrchestratorState, SalesEnablementOrchestratorConfig
from agents.sales_enablement.utilities.data_loading import (
    load_leads,
    load_sales_reps,
    load_interactions,
    load_deals,
    load_signals,
    build_leads_lookup,
    build_reps_lookup,
    build_interactions_lookup,
    build_deals_lookup,
    build_signals_lookup
)
from agents.sales_enablement.utilities.lead_prioritization import (
    prioritize_leads,
    get_top_priority_leads
)
from agents.sales_enablement.utilities.customer_analysis import (
    analyze_all_customer_needs
)
from agents.sales_enablement.utilities.follow_up_tracking import (
    extract_follow_up_actions
)
from agents.sales_enablement.utilities.rep_nudging import (
    generate_all_rep_nudges
)
from agents.sales_enablement.utilities.deal_analysis import (
    detect_stalled_deals,
    detect_at_risk_deals,
    generate_all_deal_insights
)


def goal_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Goal Node: Define the goal for the Sales Enablement Orchestrator.

    This is a simple rule-based goal definition that sets the framework.
    No dependencies - this is the entry point.
    """
    lead_id = state.get("lead_id")
    rep_id = state.get("rep_id")
    focus_area = state.get("focus_area")

    # Build focus areas based on input
    if focus_area:
        focus_areas = [focus_area]
    else:
        focus_areas = [
            "lead_prioritization",
            "customer_needs_analysis",
            "outreach_generation",
            "follow_up_coordination",
            "rep_nudging",
            "deal_insights",
            "historical_insights",
            "executive_reporting"
        ]

    goal = {
        "objective": "Enable sales team performance by prioritizing leads, analyzing customer needs, generating outreach, coordinating follow-ups, nudging reps, and surfacing actionable insights",
        "lead_id": lead_id,
        "rep_id": rep_id,
        "focus_areas": focus_areas,
        "scope": "all_leads" if not lead_id else "single_lead",
        "scope_reps": "all_reps" if not rep_id else "single_rep"
    }

    return {
        "goal": goal,
        "errors": state.get("errors", [])
    }


def planning_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan based on goal.

    This creates a step-by-step plan. Rule-based, no LLM needed.
    Only depends on goal_node.
    """
    goal = state.get("goal")

    if not goal:
        return {
            "errors": state.get("errors", []) + ["planning_node: goal is required"]
        }

    focus_areas = goal.get("focus_areas", [])

    # Build plan based on focus areas
    plan = []
    step_num = 1

    # Always start with data loading
    plan.append({
        "step": step_num,
        "name": "data_loading",
        "description": "Load all data files (leads, reps, interactions, deals, signals) and create lookup dictionaries",
        "dependencies": [],
        "outputs": ["leads", "sales_reps", "interactions", "deals", "signals", "leads_lookup", "reps_lookup", "interactions_lookup", "deals_lookup", "signals_lookup"]
    })
    step_num += 1

    # Lead prioritization (if in focus areas)
    if "lead_prioritization" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "lead_prioritization",
            "description": "Rank and prioritize leads based on intent, engagement, risk, and urgency",
            "dependencies": ["data_loading"],
            "outputs": ["prioritized_leads", "top_priority_leads"]
        })
        step_num += 1

    # Customer needs analysis (if in focus areas)
    if "customer_needs_analysis" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "customer_needs_analysis",
            "description": "Analyze customer pain points, buying signals, and objection likelihood",
            "dependencies": ["data_loading"],
            "outputs": ["customer_needs_analysis"]
        })
        step_num += 1

    # Outreach generation (if in focus areas)
    if "outreach_generation" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "outreach_generation",
            "description": "Generate personalized outreach recommendations with message drafts and timing",
            "dependencies": ["lead_prioritization", "customer_needs_analysis"],
            "outputs": ["outreach_recommendations"]
        })
        step_num += 1

    # Follow-up coordination (if in focus areas)
    if "follow_up_coordination" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "follow_up_coordination",
            "description": "Identify and track follow-up actions, detect overdue items",
            "dependencies": ["data_loading"],
            "outputs": ["follow_up_actions"]
        })
        step_num += 1

    # Rep nudging (if in focus areas)
    if "rep_nudging" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "rep_nudging",
            "description": "Generate nudges for sales reps based on overdue follow-ups, stalled deals, and high-priority leads",
            "dependencies": ["follow_up_coordination", "lead_prioritization", "deal_insights"],
            "outputs": ["rep_nudges"]
        })
        step_num += 1

    # Deal insights (if in focus areas)
    if "deal_insights" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "deal_insights",
            "description": "Detect stalled deals, at-risk deals, and opportunities",
            "dependencies": ["data_loading"],
            "outputs": ["deal_insights", "stalled_deals", "at_risk_deals"]
        })
        step_num += 1

    # Historical insights (if in focus areas)
    if "historical_insights" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "historical_insights",
            "description": "Analyze win/loss patterns from past deals to surface actionable insights",
            "dependencies": ["data_loading"],
            "outputs": ["win_patterns", "loss_patterns"]
        })
        step_num += 1

    # Executive reporting (if in focus areas)
    if "executive_reporting" in focus_areas or not focus_areas:
        plan.append({
            "step": step_num,
            "name": "executive_reporting",
            "description": "Generate pipeline summary and rep performance metrics",
            "dependencies": ["deal_insights", "lead_prioritization"],
            "outputs": ["pipeline_summary", "rep_performance_summary"]
        })
        step_num += 1

    # Always end with report generation
    plan.append({
        "step": step_num,
        "name": "report_generation",
        "description": "Generate final markdown report with all insights and recommendations",
        "dependencies": [p["name"] for p in plan if p["name"] != "report_generation"],
        "outputs": ["enablement_report", "report_file_path"]
    })

    return {
        "plan": plan,
        "errors": state.get("errors", [])
    }


def data_loading_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Data Loading Node: Orchestrate loading all data files and creating lookups.

    This node loads:
    - Leads
    - Sales Reps
    - Interactions
    - Deals
    - Signals

    And creates lookup dictionaries for fast access.
    """
    errors = state.get("errors", [])
    config = SalesEnablementOrchestratorConfig()

    try:
        # Load all data files
        leads = load_leads(config.data_dir, config.leads_file)
        sales_reps = load_sales_reps(config.data_dir, config.sales_reps_file)
        interactions = load_interactions(config.data_dir, config.interactions_file)
        deals = load_deals(config.data_dir, config.deals_file)
        signals = load_signals(config.data_dir, config.signals_file)

        # Build lookup dictionaries
        leads_lookup = build_leads_lookup(leads)
        reps_lookup = build_reps_lookup(sales_reps)
        interactions_lookup = build_interactions_lookup(interactions)
        deals_lookup = build_deals_lookup(deals)
        signals_lookup = build_signals_lookup(signals)

        return {
            "leads": leads,
            "sales_reps": sales_reps,
            "interactions": interactions,
            "deals": deals,
            "signals": signals,
            "leads_lookup": leads_lookup,
            "reps_lookup": reps_lookup,
            "interactions_lookup": interactions_lookup,
            "deals_lookup": deals_lookup,
            "signals_lookup": signals_lookup,
            "errors": errors
        }
    except FileNotFoundError as e:
        return {
            "errors": errors + [f"data_loading_node: File not found - {str(e)}"]
        }
    except json.JSONDecodeError as e:
        return {
            "errors": errors + [f"data_loading_node: Invalid JSON - {str(e)}"]
        }
    except Exception as e:
        return {
            "errors": errors + [f"data_loading_node: Unexpected error - {str(e)}"]
        }


def lead_prioritization_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Lead Prioritization Node: Rank and prioritize leads.

    Calculates priority scores for all leads and selects top N.
    """
    errors = state.get("errors", [])
    leads = state.get("leads")
    signals_lookup = state.get("signals_lookup", {})
    config = SalesEnablementOrchestratorConfig()

    if not leads:
        return {
            "errors": errors + ["lead_prioritization_node: leads required"]
        }

    try:
        # Prioritize all leads
        prioritized_leads = prioritize_leads(
            leads,
            signals_lookup,
            weights=config.priority_scoring_weights
        )

        # Get top N leads
        top_priority_leads = get_top_priority_leads(
            prioritized_leads,
            top_n=config.top_n_leads
        )

        return {
            "prioritized_leads": prioritized_leads,
            "top_priority_leads": top_priority_leads,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"lead_prioritization_node: {str(e)}"]
        }


def customer_needs_analysis_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Customer Needs Analysis Node: Analyze customer needs and pain points.

    Analyzes pain points, buying signals, objection likelihood, and product fit.
    """
    errors = state.get("errors", [])
    leads = state.get("leads")
    interactions_lookup = state.get("interactions_lookup", {})
    deals_lookup = state.get("deals_lookup", {})

    if not leads:
        return {
            "errors": errors + ["customer_needs_analysis_node: leads required"]
        }

    try:
        # Analyze all customer needs
        customer_needs_analysis = analyze_all_customer_needs(
            leads,
            interactions_lookup,
            deals_lookup
        )

        return {
            "customer_needs_analysis": customer_needs_analysis,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"customer_needs_analysis_node: {str(e)}"]
        }


def follow_up_coordination_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Follow-up Coordination Node: Identify and track follow-up actions.

    Extracts follow-up actions from interactions and tracks their status.
    """
    errors = state.get("errors", [])
    interactions = state.get("interactions", [])
    config = SalesEnablementOrchestratorConfig()

    if not interactions:
        return {
            "errors": errors + ["follow_up_coordination_node: interactions required"]
        }

    try:
        # Extract follow-up actions
        follow_up_actions = extract_follow_up_actions(
            interactions,
            follow_up_overdue_days=config.follow_up_overdue_days
        )

        return {
            "follow_up_actions": follow_up_actions,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"follow_up_coordination_node: {str(e)}"]
        }


def deal_insights_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Deal Insights Node: Detect stalled deals, at-risk deals, and opportunities.
    """
    errors = state.get("errors", [])
    deals = state.get("deals", [])
    interactions_lookup = state.get("interactions_lookup", {})
    signals_lookup = state.get("signals_lookup", {})
    config = SalesEnablementOrchestratorConfig()

    if not deals:
        return {
            "errors": errors + ["deal_insights_node: deals required"]
        }

    try:
        # Generate all deal insights
        deal_insights = generate_all_deal_insights(
            deals,
            interactions_lookup,
            signals_lookup,
            config
        )

        # Separate by type
        stalled_deals = [i for i in deal_insights if i.get("insight_type") == "stalled"]
        at_risk_deals = [i for i in deal_insights if i.get("insight_type") == "at_risk"]

        return {
            "deal_insights": deal_insights,
            "stalled_deals": stalled_deals,
            "at_risk_deals": at_risk_deals,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"deal_insights_node: {str(e)}"]
        }


def rep_nudging_node(state: SalesEnablementOrchestratorState) -> Dict[str, Any]:
    """
    Rep Nudging Node: Generate nudges for sales reps.

    Generates nudges based on:
    - Overdue follow-ups
    - Stalled deals
    - High-priority leads with no interaction
    - Deals at risk
    """
    errors = state.get("errors", [])
    follow_up_actions = state.get("follow_up_actions", [])
    stalled_deals = state.get("stalled_deals", [])
    at_risk_deals = state.get("at_risk_deals", [])
    top_priority_leads = state.get("top_priority_leads", [])
    leads_lookup = state.get("leads_lookup", {})
    interactions_lookup = state.get("interactions_lookup", {})
    config = SalesEnablementOrchestratorConfig()

    try:
        # Generate all nudges
        rep_nudges = generate_all_rep_nudges(
            follow_up_actions,
            stalled_deals,
            at_risk_deals,
            top_priority_leads,
            leads_lookup,
            interactions_lookup,
            config
        )

        return {
            "rep_nudges": rep_nudges,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"rep_nudging_node: {str(e)}"]
        }



# Test Phase 1: Foundation Nodes (Goal & Planning)

In [ ]:
"""Test Phase 1: Foundation Nodes (Goal & Planning)

Test goal_node and planning_node independently.
"""

from agents.sales_enablement.nodes import goal_node, planning_node
from config import SalesEnablementOrchestratorState


def test_goal_node():
    """Test goal_node with no input"""
    print("Testing goal_node...")

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = goal_node(state)

    assert "goal" in result, "goal_node should return 'goal'"
    assert result["goal"]["objective"] is not None, "goal should have objective"
    assert len(result["goal"]["focus_areas"]) > 0, "goal should have focus_areas"
    assert result["goal"]["scope"] == "all_leads", "scope should be 'all_leads' when no lead_id"

    print(f"✅ goal_node test passed!")
    print(f"   Objective: {result['goal']['objective'][:60]}...")
    print(f"   Focus areas: {len(result['goal']['focus_areas'])} areas")
    print(f"   Scope: {result['goal']['scope']}")
    print()


def test_goal_node_with_lead_id():
    """Test goal_node with specific lead_id"""
    print("Testing goal_node with lead_id...")

    state: SalesEnablementOrchestratorState = {
        "lead_id": "L-001",
        "errors": []
    }

    result = goal_node(state)

    assert result["goal"]["lead_id"] == "L-001", "goal should include lead_id"
    assert result["goal"]["scope"] == "single_lead", "scope should be 'single_lead'"

    print(f"✅ goal_node with lead_id test passed!")
    print(f"   Lead ID: {result['goal']['lead_id']}")
    print(f"   Scope: {result['goal']['scope']}")
    print()


def test_goal_node_with_focus_area():
    """Test goal_node with specific focus_area"""
    print("Testing goal_node with focus_area...")

    state: SalesEnablementOrchestratorState = {
        "focus_area": "lead_prioritization",
        "errors": []
    }

    result = goal_node(state)

    assert result["goal"]["focus_areas"] == ["lead_prioritization"], "focus_areas should match input"

    print(f"✅ goal_node with focus_area test passed!")
    print(f"   Focus areas: {result['goal']['focus_areas']}")
    print()


def test_planning_node():
    """Test planning_node with goal"""
    print("Testing planning_node...")

    state: SalesEnablementOrchestratorState = {
        "goal": {
            "objective": "Test objective",
            "focus_areas": ["lead_prioritization", "outreach_generation"]
        },
        "errors": []
    }

    result = planning_node(state)

    assert "plan" in result, "planning_node should return 'plan'"
    assert len(result["plan"]) > 0, "plan should have steps"

    # Check that plan includes data_loading (always first)
    assert result["plan"][0]["name"] == "data_loading", "First step should be data_loading"

    # Check that plan includes requested focus areas
    plan_names = [step["name"] for step in result["plan"]]
    assert "lead_prioritization" in plan_names, "plan should include lead_prioritization"
    assert "outreach_generation" in plan_names, "plan should include outreach_generation"

    # Check that plan ends with report_generation
    assert result["plan"][-1]["name"] == "report_generation", "Last step should be report_generation"

    print(f"✅ planning_node test passed!")
    print(f"   Total steps: {len(result['plan'])}")
    print(f"   Steps: {[step['name'] for step in result['plan']]}")
    print()


def test_planning_node_without_goal():
    """Test planning_node error handling when goal is missing"""
    print("Testing planning_node error handling...")

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = planning_node(state)

    assert "errors" in result, "planning_node should return errors"
    assert len(result["errors"]) > 0, "should have error when goal is missing"
    assert "goal is required" in result["errors"][0], "error should mention goal"

    print(f"✅ planning_node error handling test passed!")
    print(f"   Error: {result['errors'][0]}")
    print()


def test_goal_and_planning_chain():
    """Test goal_node → planning_node chain"""
    print("Testing goal_node → planning_node chain...")

    # Start with empty state
    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    # Run goal_node (merge result into state)
    goal_result = goal_node(state)
    state.update(goal_result)
    assert "goal" in state, "goal should be in state after goal_node"

    # Run planning_node (merge result into state)
    plan_result = planning_node(state)
    state.update(plan_result)
    assert "plan" in state, "plan should be in state after planning_node"
    assert len(state["errors"]) == 0, "should have no errors in successful chain"

    print(f"✅ goal_node → planning_node chain test passed!")
    print(f"   Goal objective: {state['goal']['objective'][:50]}...")
    print(f"   Plan steps: {len(state['plan'])}")
    print()


def test_data_loading_node():
    """Test data_loading_node"""
    print("Testing data_loading_node...")

    from agents.sales_enablement.nodes import data_loading_node

    state: SalesEnablementOrchestratorState = {
        "errors": []
    }

    result = data_loading_node(state)

    assert "leads" in result, "data_loading_node should return 'leads'"
    assert "sales_reps" in result, "data_loading_node should return 'sales_reps'"
    assert "interactions" in result, "data_loading_node should return 'interactions'"
    assert "deals" in result, "data_loading_node should return 'deals'"
    assert "signals" in result, "data_loading_node should return 'signals'"
    assert "leads_lookup" in result, "data_loading_node should return 'leads_lookup'"
    assert "reps_lookup" in result, "data_loading_node should return 'reps_lookup'"
    assert "interactions_lookup" in result, "data_loading_node should return 'interactions_lookup'"
    assert "deals_lookup" in result, "data_loading_node should return 'deals_lookup'"
    assert "signals_lookup" in result, "data_loading_node should return 'signals_lookup'"
    assert len(result["errors"]) == 0, "should have no errors"

    # Verify data loaded correctly
    assert len(result["leads"]) == 20, "should load 20 leads"
    assert len(result["sales_reps"]) == 4, "should load 4 reps"
    assert len(result["interactions"]) == 12, "should load 12 interactions"
    assert len(result["deals"]) == 15, "should load 15 deals"
    assert len(result["signals"]) == 20, "should load 20 signals"

    # Verify lookups work
    assert "L-001" in result["leads_lookup"], "leads_lookup should contain L-001"
    assert "SR-01" in result["reps_lookup"], "reps_lookup should contain SR-01"

    print(f"✅ data_loading_node test passed!")
    print(f"   Loaded: {len(result['leads'])} leads, {len(result['sales_reps'])} reps, {len(result['interactions'])} interactions, {len(result['deals'])} deals, {len(result['signals'])} signals")
    print(f"   Lookups: {len(result['leads_lookup'])} leads, {len(result['reps_lookup'])} reps, {len(result['interactions_lookup'])} leads with interactions, {len(result['deals_lookup'])} leads with deals")
    print()

    return result


def test_lead_prioritization_node():
    """Test lead_prioritization_node"""
    print("Testing lead_prioritization_node...")

    from agents.sales_enablement.nodes import lead_prioritization_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then prioritize
    result = lead_prioritization_node(state)

    assert "prioritized_leads" in result, "should return prioritized_leads"
    assert "top_priority_leads" in result, "should return top_priority_leads"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["prioritized_leads"]) == 20, "should prioritize all 20 leads"
    assert len(result["top_priority_leads"]) == 10, "should return top 10 leads"
    assert result["prioritized_leads"][0]["priority_score"] >= result["prioritized_leads"][-1]["priority_score"], "should be sorted"

    print(f"✅ lead_prioritization_node test passed!")
    print(f"   Prioritized: {len(result['prioritized_leads'])} leads")
    print(f"   Top {len(result['top_priority_leads'])} leads:")
    for i, lead in enumerate(result["top_priority_leads"][:3], 1):
        print(f"     {i}. {lead['lead_id']} - {lead['company_name']} (Score: {lead['priority_score']:.2f})")
    print()


def test_customer_needs_analysis_node():
    """Test customer_needs_analysis_node"""
    print("Testing customer_needs_analysis_node...")

    from agents.sales_enablement.nodes import customer_needs_analysis_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then analyze
    result = customer_needs_analysis_node(state)

    assert "customer_needs_analysis" in result, "should return customer_needs_analysis"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["customer_needs_analysis"]) == 20, "should analyze all 20 leads"
    assert "buying_signals" in result["customer_needs_analysis"][0], "should have buying_signals"
    assert "product_fit" in result["customer_needs_analysis"][0], "should have product_fit"

    print(f"✅ customer_needs_analysis_node test passed!")
    print(f"   Analyzed: {len(result['customer_needs_analysis'])} leads")
    sample = result["customer_needs_analysis"][0]
    print(f"   Sample: {sample['lead_id']} - Product fit: {sample['product_fit']}, Buying signals: {len(sample['buying_signals'])}")
    print()


def test_follow_up_coordination_node():
    """Test follow_up_coordination_node"""
    print("Testing follow_up_coordination_node...")

    from agents.sales_enablement.nodes import follow_up_coordination_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then coordinate follow-ups
    result = follow_up_coordination_node(state)

    assert "follow_up_actions" in result, "should return follow_up_actions"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["follow_up_actions"]) > 0, "should find some follow-ups"
    assert "status" in result["follow_up_actions"][0], "should have status"

    print(f"✅ follow_up_coordination_node test passed!")
    print(f"   Follow-up actions: {len(result['follow_up_actions'])}")
    overdue = [f for f in result["follow_up_actions"] if f.get("status") == "overdue"]
    print(f"   Overdue: {len(overdue)}")
    print()


def test_deal_insights_node():
    """Test deal_insights_node"""
    print("Testing deal_insights_node...")

    from agents.sales_enablement.nodes import deal_insights_node, data_loading_node

    # First load data
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))

    # Then analyze deals
    result = deal_insights_node(state)

    assert "deal_insights" in result, "should return deal_insights"
    assert "stalled_deals" in result, "should return stalled_deals"
    assert "at_risk_deals" in result, "should return at_risk_deals"
    assert len(result["errors"]) == 0, "should have no errors"

    print(f"✅ deal_insights_node test passed!")
    print(f"   Total insights: {len(result['deal_insights'])}")
    print(f"   Stalled deals: {len(result['stalled_deals'])}")
    print(f"   At-risk deals: {len(result['at_risk_deals'])}")
    print()


def test_rep_nudging_node():
    """Test rep_nudging_node"""
    print("Testing rep_nudging_node...")

    from agents.sales_enablement.nodes import (
        rep_nudging_node, data_loading_node, lead_prioritization_node,
        follow_up_coordination_node, deal_insights_node
    )

    # Build up state
    state: SalesEnablementOrchestratorState = {"errors": []}
    state.update(data_loading_node(state))
    state.update(lead_prioritization_node(state))
    state.update(follow_up_coordination_node(state))
    state.update(deal_insights_node(state))

    # Then generate nudges
    result = rep_nudging_node(state)

    assert "rep_nudges" in result, "should return rep_nudges"
    assert len(result["errors"]) == 0, "should have no errors"
    assert len(result["rep_nudges"]) > 0, "should generate some nudges"
    assert "nudge_type" in result["rep_nudges"][0], "should have nudge_type"
    assert "message" in result["rep_nudges"][0], "should have message"

    print(f"✅ rep_nudging_node test passed!")
    print(f"   Generated {len(result['rep_nudges'])} nudges")

    # Show nudge types
    nudge_types = {}
    for nudge in result["rep_nudges"]:
        nudge_type = nudge.get("nudge_type", "unknown")
        nudge_types[nudge_type] = nudge_types.get(nudge_type, 0) + 1

    print(f"   Nudge types: {nudge_types}")
    if result["rep_nudges"]:
        print(f"   Sample nudge: {result['rep_nudges'][0]['message'][:60]}...")
    print()


if __name__ == "__main__":
    print("=" * 60)
    print("Phase 1, 2, 3 & 4: Foundation + Data Loading + Analysis + Follow-up Nodes - Test Suite")
    print("=" * 60)
    print()

    test_goal_node()
    test_goal_node_with_lead_id()
    test_goal_node_with_focus_area()
    test_planning_node()
    test_planning_node_without_goal()
    test_goal_and_planning_chain()
    test_data_loading_node()
    test_lead_prioritization_node()
    test_customer_needs_analysis_node()
    test_follow_up_coordination_node()
    test_deal_insights_node()
    test_rep_nudging_node()

    print("=" * 60)
    print("✅ All Phase 1, 2, 3 & 4 tests passed!")
    print("=" * 60)



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_007_TEMPLATE copy % python test_sales_enablement_phase1.py
============================================================
Phase 1, 2, 3 & 4: Foundation + Data Loading + Analysis + Follow-up Nodes - Test Suite
============================================================

Testing goal_node...
✅ goal_node test passed!
   Objective: Enable sales team performance by prioritizing leads, analyzi...
   Focus areas: 8 areas
   Scope: all_leads

Testing goal_node with lead_id...
✅ goal_node with lead_id test passed!
   Lead ID: L-001
   Scope: single_lead

Testing goal_node with focus_area...
✅ goal_node with focus_area test passed!
   Focus areas: ['lead_prioritization']

Testing planning_node...
✅ planning_node test passed!
   Total steps: 4
   Steps: ['data_loading', 'lead_prioritization', 'outreach_generation', 'report_generation']

Testing planning_node error handling...
✅ planning_node error handling test passed!
   Error: planning_node: goal is required

Testing goal_node → planning_node chain...
✅ goal_node → planning_node chain test passed!
   Goal objective: Enable sales team performance by prioritizing lead...
   Plan steps: 10

Testing data_loading_node...
✅ data_loading_node test passed!
   Loaded: 20 leads, 4 reps, 12 interactions, 15 deals, 20 signals
   Lookups: 20 leads, 4 reps, 9 leads with interactions, 15 leads with deals

Testing lead_prioritization_node...
✅ lead_prioritization_node test passed!
   Prioritized: 20 leads
   Top 10 leads:
     1. L-020 - Orion Aerospace (Score: 86.60)
     2. L-003 - Apex Manufacturing (Score: 85.50)
     3. L-011 - Atlas Freight (Score: 83.00)

Testing customer_needs_analysis_node...
✅ customer_needs_analysis_node test passed!
   Analyzed: 20 leads
   Sample: L-001 - Product fit: 0.63, Buying signals: 1

Testing follow_up_coordination_node...
✅ follow_up_coordination_node test passed!
   Follow-up actions: 7
   Overdue: 7

Testing deal_insights_node...
✅ deal_insights_node test passed!
   Total insights: 11
   Stalled deals: 2
   At-risk deals: 5

Testing rep_nudging_node...
✅ rep_nudging_node test passed!
   Generated 24 nudges
   Nudge types: {'follow_up_due': 7, 'stalled_deal': 2, 'high_priority_lead': 10, 'deal_at_risk': 5}
   Sample nudge: ⚠️ Follow-up with Northstar Logistics (L-001) is overdue. Yo...

============================================================
✅ All Phase 1, 2, 3 & 4 tests passed!
============================================================
